In [287]:
# !pip install opencv-python
# !pip install tqdm

In [288]:
import numpy as np
import pandas as pd
import math
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# PARAMETERS

In [289]:
TRAINING_A_DIR = './NumtaDB/training-a/'
TRAINING_A_CSV = './NumtaDB/training-a.csv'

TRAINING_B_DIR = './NumtaDB/training-b/'
TRAINING_B_CSV = './NumtaDB/training-b.csv'

TRAINING_C_DIR = './NumtaDB/training-c/'
TRAINING_C_CSV = './NumtaDB/training-c.csv'

TRAINING_D_DIR = './NumtaDB/training-d/'
TRAINING_D_CSV = './NumtaDB/training-d.csv'

TRAINING_E_DIR = './NumtaDB/training-e/'
TRAINING_E_CSV = './NumtaDB/training-e.csv'

DONT_INVERT_IMAGE_DIR = "training-e"



In [290]:
MODEL_INIT_FILE = 'model_desc.txt'
IMAGE_DATASET_DIRS = [TRAINING_A_DIR, TRAINING_B_DIR, TRAINING_C_DIR]
CSV_FILES = [TRAINING_A_CSV, TRAINING_B_CSV, TRAINING_C_CSV]
MINI_BATCH_SIZE = 64
IMAGE_DIM = 28 # Height and width of the image
X_train = []
y_train = []

# PROCESS DATASET

In [291]:
def read_image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    # Resizing the image
    img = cv2.resize(img,(IMAGE_DIM, IMAGE_DIM))
    # Displaying the image
    # plt.imshow(img, cmap='gray')
    # plt.show()
    # print(img.shape)
    img = np.array(img)
    img = img.astype('float32')
    # Inverting the image
    if DONT_INVERT_IMAGE_DIR not in path:
        img = 255 - img
    # dilation
    kernel = np.ones((2,2), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    # plt.imshow(img, cmap='gray')
    # plt.show()
    # print(img.shape)
    img /= 255
    # reshaping the image
    img = img.reshape(IMAGE_DIM, IMAGE_DIM, 1) # 1 for grayscale
    return img

In [292]:
# https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files
class CustomImageDataset:
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, self.img_labels.columns.get_loc('filename')])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, self.img_labels.columns.get_loc('digit')] # 3 is the column index of the label
        #one hot encoding
        label = np.eye(10)[label]
        
        # if self.transform:
        #     image = self.transform(image)
        # if self.target_transform:
        #     label = self.target_transform(label)
        return image, label

In [293]:
class CustomDataLoader:
    def __init__(self, dataset, batch_size=32, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.current_idx = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current_idx >= len(self.dataset):
            raise StopIteration
        
        # Get the next batch.
        if self.current_idx + self.batch_size > len(self.dataset):
            batch = [self.dataset[i] for i in range(self.current_idx, len(self.dataset))]
            self.current_idx = len(self.dataset)
        else:
            batch = [self.dataset[i] for i in range(self.current_idx, self.current_idx + self.batch_size)]
            self.current_idx += self.batch_size
        

        if self.shuffle:
            np.random.shuffle(batch)

        images, labels = zip(*batch)
        images = np.stack(images)
        labels = np.stack(labels)

        return images, labels

# READ DATASET

In [294]:
print("Reading Images...")
for i, IMAGE_DATASET_DIR in enumerate(IMAGE_DATASET_DIRS):
    CSV_FILE = CSV_FILES[i]
    dataset = CustomImageDataset(annotations_file= CSV_FILE, img_dir=IMAGE_DATASET_DIR)
    dataloader = CustomDataLoader(dataset, batch_size=MINI_BATCH_SIZE, shuffle=False)
    for images, labels in dataloader:
        # print(images.shape, labels.shape)
        X_train.append(images)
        y_train.append(labels)
        # break
    # break
            
print("Reading Images Completed")

X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)
print(X_train.shape, y_train.shape)

Reading Images...
Reading Images Completed
(44359, 28, 28, 1) (44359, 10)


# CLASSES FOR CNN

<h2>SOFTMAX LAYER</h2>

In [295]:
# done
# https://stats.stackexchange.com/questions/304758/softmax-overflow?fbclid=IwAR0jL84MQqvY2Xk_CeBwaUM7kwtRBvV6O7yKtJhtcvGtawp0BwhKsZ4Buwk
class Softmax_Layer:
    def __init__(self):
        self.layer_type = 'Softmax'
    
    def __str__(self):
        return f"{self.layer_type} Layer"
    
    def forward(self, X):
        x_max = np.max(X, axis=1)
        x_max = x_max.reshape(x_max.shape[0], 1)
        Z = np.exp(X-x_max)
        # Z = np.exp(X)
        sum = np.einsum('ij->i', Z)
        sum = sum.reshape(sum.shape[0], 1)
        return Z / sum
    
    def backward(self, dZ, learning_rate=0.0001):
        return np.copy(dZ)

<h2>ReLU ACTIVATION </h2>

In [296]:
# done
class ReLU_Activation:
    def __init__(self):
        self.layer_type = 'ReLU'
    
    def __str__(self):
        return f"{self.layer_type} Activation"
    
    def forward(self, X):
        self.X = X
        Z = np.copy(X)
        Z[Z < 0] = 0
        return Z
    
    def backward(self, dZ, learning_rate=0.0001):
        dX = np.copy(self.X)
        dX[dX < 0] = 0
        dX[dX > 0] = 1
        return dX * dZ

<h2>FULLY CONNECTED LAYER</h2>

In [297]:
# done
class Fully_Connected_Layer:
    def __init__(self, output_dim):
        self.output_dim = output_dim
        self.W = None
        self.b = None

    def __str__(self):
        return f"Fully Connected Layer(output_dim={self.output_dim})"
    
    def forward(self, X):
        self.X = X

        if self.W is None:
            self.W = np.random.randn(X.shape[1], self.output_dim) * math.sqrt(2 / X.shape[0])
        
        if self.b is None:
            self.b = np.zeros((1, self.output_dim))

        Z = np.einsum('ij,jk->ik', X, self.W) + self.b
        
        return Z
    
    def backward(self, dZ, learning_rate=0.0001):
        dW = np.einsum('ij,ik->jk', self.X, dZ) / self.X.shape[0]
        db = np.einsum('ij->j', dZ) / self.X.shape[0] 
        dX = np.einsum('ij,jk->ik', dZ, self.W.T)

        self.W = self.W - learning_rate * dW
        self.b = self.b - learning_rate * db

        return dX

<h2>FLATENNING LAYER</h2>

In [298]:
class Flatenning_Layer:
    def __init__(self):
        self.layer_type = 'Flatten'
    
    def __str__(self):
        return f"{self.layer_type} Layer"
    
    def forward(self, X):
        self.input_shape = X.shape
        # print(f"input shape : {X.shape}")
        # print(f"output shape : {X.reshape((X.shape[0], -1)).shape}")
        return X.reshape((X.shape[0], -1)) # check here
    
    def backward(self, dZ, learning_rate=0.0001):
        dX = np.copy(dZ)
        return dX.reshape(self.input_shape) # check here

<h3> WINDOWS: as_strided </h3>

In [299]:
def getWindows(input, output_size, kernel_size, padding=0, stride=1, dilate=0):
    working_input = input
    working_pad = padding
    
    # dilate the input if necessary
    if dilate != 0:
        working_input = np.insert(working_input, range(1, input.shape[1]), 0, axis=1)
        working_input = np.insert(working_input, range(1, input.shape[2]), 0, axis=2)

    # pad the input if necessary
    if working_pad != 0:
        working_input = np.pad(working_input, pad_width=((0,), (working_pad,), (working_pad,), (0,)), mode='constant', constant_values=(0.,))

    in_b, out_h, out_w, in_c = output_size
    out_b, _, _, out_c = input.shape
    batch_str, kern_h_str, kern_w_str, channel_str = working_input.strides

    return np.lib.stride_tricks.as_strided(
        working_input,
        (out_b, out_h, out_w, kernel_size, kernel_size, out_c),
        (batch_str, stride * kern_h_str, stride * kern_w_str, kern_h_str, kern_w_str, channel_str)
    )

<h1>MAX POOLING</h1>

In [300]:

# class Max_Pooling:
#     def __init__(self, filter_dim, stride):
#         self.layer_type = 'Max Pooling'
#         self.filter_dim = filter_dim
#         self.stride = stride
#         # self.X = None
#         # self.Z_Max_idx = None
    
#     def __str__(self):
#         return f"{self.layer_type} (filter_dim={self.filter_dim}, stride={self.stride})"
    

#     def forward(self, X):
#         self.X_shape = X.shape
#         n, h, w, c = X.shape
#         new_h = (h - self.filter_dim) // self.stride + 1
#         new_w = (w - self.filter_dim) // self.stride + 1
        
#         X_strided = getWindows(X, (n, new_h, new_w, c), self.filter_dim, stride=self.stride)
        
#         self.X_strided_shape = X_strided.shape

#         Z = X_strided.max(axis=(3, 4))
        
#         self.Z_Max_idx = np.zeros(Z.shape, dtype=np.int32)

#         for i in range(self.filter_dim):
#             for j in range(self.filter_dim):
#                 self.Z_Max_idx += (X_strided[:, :, :, i, j, :] == Z)

#         return Z

#     def backward(self, dZ, learning_rate=0.0001):
#         # print(dZ.shape)
#         n, h_new, w_new, c = dZ.shape
#         dX = np.zeros(self.X_strided_shape)
#         dZ_flat = dZ.ravel()

#         for i in range(dZ_flat.shape[0]):
#             max_idx = np.unravel_index(self.Z_Max_idx.flat[i], (n, h_new, w_new, self.filter_dim, self.filter_dim))
#             # print(max_idx)
#             dX[max_idx + (slice(None),)] = dZ_flat[i]
#         # print(self.X_shape)
#         # print(dX.shape)
#         dX = dX.reshape(self.X_shape)
#         return dX


In [301]:
# new max pooling
class Max_Pooling:
    def __init__(self, filter_dim, stride):
        self.layer_type = 'Max Pooling'
        self.filter_dim = filter_dim
        self.stride = stride
        # self.X = None
        # self.Z_Max_idx = None
    
    def __str__(self):
        return f"{self.layer_type} (filter_dim={self.filter_dim}, stride={self.stride})"
    

    def forward(self, X):
        self.X_shape = X.shape
        n, h, w, c = X.shape
        out_h = (h - self.filter_dim) // self.stride + 1
        out_w = (w - self.filter_dim) // self.stride + 1
        
        X_strided = np.lib.stride_tricks.as_strided(
            X,
            shape=(n, out_h, out_w, self.filter_dim, self.filter_dim, c),
            strides=(X.strides[0], self.stride * X.strides[1], self.stride * X.strides[2], X.strides[1], X.strides[2], X.strides[3]),
            writeable=False
        )
        
        out = np.max(X_strided, axis=(3, 4))

        maxs = out.repeat(self.filter_dim, axis=1).repeat(self.filter_dim, axis=2)

        x_window = X[:, :out_h * self.stride, :out_w * self.stride, :]

        mask = np.equal(x_window, maxs).astype(int)

        # self.X = X
        self.mask = mask
        
        return out

    def backward(self, dA_prev, learning_rate=0.0001):
        # print(dA_prev.shape)
        mask = self.mask
        dA = dA_prev.repeat(self.filter_dim, axis=1).repeat(self.filter_dim, axis=2)
        dA = np.multiply(dA, mask)
        pad = np.zeros(self.X_shape)
        pad[:, :dA.shape[1], :dA.shape[2], :] = dA
        
        return pad  



<h1>CONVOLUTION</h1>

In [302]:
# https://medium.com/@mayank.utexas/backpropagation-for-convolution-with-strides-8137e4fc2710
# https://blog.ca.meron.dev/Vectorized-CNN/?fbclid=IwAR2GWeVd2AnOGLJrpDqNAFC6F2m5dhrNamB-km8y7D-0TKm4K7Uz1W-2L6Y
class Convolution:
    def __init__(self, num_output_channels, filter_dim, stride=1, padding=0):
        self.layer_type = 'Convolution'
        self.num_output_channels = num_output_channels
        self.filter_dim = filter_dim
        self.stride = stride
        self.padding = padding
        self.W = None
        self.b = None
    
    def __str__(self):
        return f"{self.layer_type} (num_output_channels={self.num_output_channels}, filter_dim={self.filter_dim}, stride={self.stride}, padding={self.padding})"
    
    def forward(self, X):
        self.X = X
        n, h, w, c = X.shape
        out_h = (h - self.filter_dim + 2*self.padding) // self.stride + 1
        out_w = (w - self.filter_dim + 2*self.padding) // self.stride + 1

        if self.W is None:
            self.W = np.random.randn(self.num_output_channels, self.filter_dim, self.filter_dim, X.shape[3]) * math.sqrt(2 / X.shape[0])
        if self.b is None:
            self.b = np.zeros((self.num_output_channels))
        
        X_strided = getWindows(X, (n, out_h, out_w, c), self.filter_dim, self.padding, self.stride)
        self.X_strided = X_strided

        Z = np.einsum('ijklmn,olmn->ijko', X_strided, self.W) + self.b

        return Z
    
        
    def backward(self, dZ, learning_rate=0.0001):
        
        padding = self.filter_dim - 1 if self.padding == 0 else self.padding
        dout_windows = getWindows(dZ, self.X.shape, self.filter_dim, padding=padding, stride=1, dilate=self.stride - 1)

        rot_kern = np.rot90(self.W, 2, axes=(1, 2))

        db = np.einsum('mijc->c', dZ)/dZ.shape[0]
        dW = np.einsum('ijkmno,ijkl->lmno', self.X_strided, dZ)/dZ.shape[0]
        dX = np.einsum('mhwijc,cijk->mhwk', dout_windows, rot_kern)
    
        self.W = self.W - learning_rate * dW
        self.b = self.b - learning_rate * db
        return dX

<h1>MODEL</h1>

In [303]:
class Model:
    def __init__(self, filePath):
        self.layers = []
        self.filePath = filePath
        self.build_model()

    def __str__(self):
        string = 'MODEL DETAILS:\n\n'
        for i, layer in enumerate(self.layers):
            string += f"Layer {i+1}: {layer}\n"
        return string
    
    def build_model(self):
        #check if file exists
        if not os.path.exists(self.filePath):
            print('File does not exist')
            return
        with open(self.filePath, 'r') as file:
            lines = file.readlines()
            for line in lines:
                if line.startswith('#'):
                    continue

                line = line.strip()
                
                if line == '':
                    continue

                line_split = line.split(' ')
                layer_name = str(line_split[0]).upper()
                
                if layer_name == 'FC':
                    output_dim = int(line_split[1])
                    self.layers.append(Fully_Connected_Layer(output_dim))

                elif layer_name == 'CONV':
                    num_output_channels = int(line_split[1])
                    filter_dim = int(line_split[2])
                    stride = int(line_split[3])
                    padding = int(line_split[4])
                    self.layers.append(Convolution(num_output_channels, filter_dim, stride, padding))

                elif layer_name == 'MAXPOOL':
                    filter_dim = int(line_split[1])
                    stride = int(line_split[2])
                    self.layers.append(Max_Pooling(filter_dim, stride))

                elif layer_name == 'FLATTEN':
                    self.layers.append(Flatenning_Layer())

                elif layer_name == 'RELU':
                    self.layers.append(ReLU_Activation())

                elif layer_name == 'SOFTMAX':
                    self.layers.append(Softmax_Layer())
                
                else:
                    print('Invalid layer name')
                    return
        
    def forward(self, X):
        for layer in self.layers:
            # print("forward : ", layer)
            X = layer.forward(X)
        return X
    
    def backward(self, dZ, learning_rate=0.0001):
        for layer in reversed(self.layers):
            # print("Backward : ",layer)
            dZ = layer.backward(dZ, learning_rate)
        return dZ
    
    def train(self, X, Y, X_val, y_val, learning_rate=0.0001, epochs=10, batch_size=64):
        epochs = int(epochs)
        results = []
        for epoch in tqdm(range(epochs)):
            for i in range(0, X.shape[0], batch_size):
                X_batch = X[i:i+batch_size]
                Y_batch = Y[i:i+batch_size]
                Z = self.forward(X_batch)
                dZ = Z - Y_batch
                self.backward(dZ, learning_rate)
                # print(f"progress: {i+batch_size}/{X.shape[0]} completed. Loss: {self.loss(X_batch, Y_batch)}")
            train_loss = self.loss(X, Y)
            train_acc, y_pred = self.evaluate(X, Y)
            y_true = np.argmax(Y, axis=1)
            F1_score = f1_score(y_true, y_pred, average='macro')
            
            val_loss = self.loss(X_val, y_val)
            val_acc, _ = self.evaluate(X_val, y_val)
            results.append([train_loss, train_acc, val_loss, val_acc, F1_score])
            print(f"Epoch {epoch+1} completed. Train Loss: {train_loss}, Train Accuracy: {train_acc}, f1 score: {F1_score},\nValidation Loss: {val_loss} ,Validation Accuracy: {val_acc}")
            if val_acc>90:
                print("\n==================>>>Early Stopping. Validation Accuracy > 90% <<<=======================\n")
                break
        return results

    def predict(self, X):
        Z = self.forward(X)
        return np.argmax(Z, axis=1)
    
    def evaluate(self, X, Y):
        Y_pred = self.predict(X)
        Y_true = np.argmax(Y, axis=1)
        return np.sum(Y_pred == Y_true) / len(Y_true) * 100, Y_pred
    
    def loss(self, X, Y):
        Z = self.forward(X)
        return -np.mean(Y * np.log(Z))

    

# BUILD MODEL

In [304]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=False)
print(X_train.shape, y_train.shape,"\n", X_val.shape, y_val.shape)

model = Model(MODEL_INIT_FILE)
print(model)

results = model.train(X_train, y_train, X_val, y_val, learning_rate=0.01, epochs=30, batch_size=64)

(39923, 28, 28, 1) (39923, 10) 
 (4436, 28, 28, 1) (4436, 10)
MODEL DETAILS:

Layer 1: Convolution (num_output_channels=6, filter_dim=5, stride=1, padding=2)
Layer 2: ReLU Activation
Layer 3: Max Pooling (filter_dim=2, stride=2)
Layer 4: Convolution (num_output_channels=16, filter_dim=5, stride=1, padding=0)
Layer 5: ReLU Activation
Layer 6: Max Pooling (filter_dim=2, stride=2)
Layer 7: Flatten Layer
Layer 8: Fully Connected Layer(output_dim=120)
Layer 9: ReLU Activation
Layer 10: Fully Connected Layer(output_dim=84)
Layer 11: ReLU Activation
Layer 12: Fully Connected Layer(output_dim=10)
Layer 13: Softmax Layer



  3%|▎         | 1/30 [04:23<2:07:12, 263.19s/it]

Epoch 1 completed. Train Loss: 0.15283579246997867, Train Accuracy: 48.47832076747739, f1 score: 0.48428696365343515,
Validation Loss: 0.14491133721314428 ,Validation Accuracy: 50.40577096483319


  7%|▋         | 2/30 [08:58<2:06:04, 270.17s/it]

Epoch 2 completed. Train Loss: 0.1856036929405518, Train Accuracy: 51.39393332164417, f1 score: 0.5216813215170091,
Validation Loss: 0.09871485175994658 ,Validation Accuracy: 67.04238052299368


...

Epoch 28 completed. Train Loss: 0.20834202392891996, Train Accuracy: 63.26428374621146, f1 score: 0.6640757856158161,
Validation Loss: 0.03608970397016151 ,Validation Accuracy: 88.32281334535618


 97%|█████████▋| 29/30 [2:04:37<04:11, 251.77s/it]

Epoch 29 completed. Train Loss: 0.20566468683245354, Train Accuracy: 63.70763720161311, f1 score: 0.6683348763747554,
Validation Loss: 0.03587890350147402 ,Validation Accuracy: 88.48061316501352


100%|██████████| 30/30 [2:08:42<00:00, 257.41s/it]

Epoch 30 completed. Train Loss: 0.20732508907029476, Train Accuracy: 63.76775292437943, f1 score: 0.6688988544701651,
Validation Loss: 0.035938723964304864 ,Validation Accuracy: 88.48061316501352


# SAVE MODEL

In [305]:
# to save output of model
if len(results) != 0:
    with open("1705027_output.txt", "w") as file:
        for result in results:
            file.write(f"{result[0]} {result[1]} {result[2]} {result[3]}\n")
# to save model
with open('1705027_model.pickle', 'wb') as file:
    pickle.dump(model, file)

# TESTING

In [307]:
TEST_MODEL_PATH = './saved_model/model_cnn.pkl'
TEST_DIR = TRAINING_D_DIR
TEST_CSV = TRAINING_D_CSV
TEST_DATA_PATH = TEST_DIR
TEST_DATA_LABEL_PATH = TEST_CSV

test_dataset = CustomImageDataset(annotations_file= TEST_DATA_LABEL_PATH, img_dir=TEST_DATA_PATH)
test_dataloader = CustomDataLoader(test_dataset, batch_size=128, shuffle=False)

X_test = []
y_test = []

for images, labels in test_dataloader:
    # print(images.shape, labels.shape)
    X_test.append(images)
    y_test.append(labels)

X_test = np.concatenate(X_test)
y_test = np.concatenate(y_test)
print(X_test.shape, y_test.shape)
# print(model.predict(images))
acc, _ = model.evaluate(X_test, y_test)
print("Test Accuracy: ", acc)

(10908, 28, 28, 1) (10908, 10)
Test Accuracy:  75.33920058672534
